<a href="https://colab.research.google.com/github/kwanglo/lookinbody_test/blob/master/code/inbody_inceptionV3_mealCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [199]:
#구글 드라이브와 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [200]:
#사용 라이브러리 import
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import datetime
import glob  # finding image paths
from skimage.io import imread  # reading images
from skimage.transform import resize  # resizing images
import PIL
import PIL.Image

import tensorflow as tf
!pip -q install tensorflow-hub
import tensorflow_hub as hub

import time
from IPython.display import Image, display

In [201]:
#공유폴더에서 해당 directory로 바로가기 추가 후 진행
path = '/gdrive/My Drive/Colab Notebooks/사진 자료'

### 사용자 및 사진 정보 추출

1. 디렉토리에서 폴더 이름 추출<br>
2. 각 폴더에서 사진정보에 따라 촬영날짜, 시간 추출<br>
3. 시간 데이터에 따른 식사 카테고리 구분

In [270]:
#사용자 정보 추출 후 .DS_Store 삭제
user_name = pd.DataFrame(os.listdir(path))
user_name.columns = ['userID']
user_name = user_name[user_name.userID != '.DS_Store']
#필요 시 userID 순서대로 정렬 ==> 불필요한 경우 주석처리 가능
user_name.sort_values(by=['userID'], axis=0, inplace=True)
user_name = user_name.reset_index(drop=True)
#전체 사용자의 userID순서대로 정렬된 path list 생성
path_temp = path + "/" + user_name[['userID'][0]]

In [271]:
#각 사용자별 사진 이름을 포함하는 테이블 생성 - 전체 데이터
df_table = pd.DataFrame()
#df_table.columns = ['file_name','userID','date_info','meal_category'] #최종 column

for i in range(0,len(path_temp)):
  #각 userID에 따라 파일명 구하기
  file_name_temp = pd.DataFrame(os.listdir(path_temp[i]))
  file_name_temp.columns = ['col_temp']
  #.DS_Store 파일 삭제
  file_name_temp = file_name_temp[file_name_temp.col_temp != '.DS_Store']  
  #임시 테이블 생성
  df_temp = pd.DataFrame()
  #파일명과 유저ID 입력
  df_temp['file_name'] = file_name_temp['col_temp']
  df_temp['userID'] = user_name[['userID'][0]][i]
  #파일명에서 시간정보 추출
  df_temp['date_info'] = df_temp['file_name'].str.slice(stop=12)
  df_temp['date_info'] = pd.to_datetime(df_temp.date_info, format='%Y%m%d%H%M', errors='ignore')
  #식사 시간 기준으로 meal_category 생성
  #06~10시 아침
  #11~16시 점심
  #17~21시 저녁
  #21~05시 야식
  df_temp['meal_category'] = '미분류' #카테고리 기본값 설정
  #기준 시각에 따라 식사 카테고리 분류
  df_temp.loc[(df_temp['date_info'].dt.hour > 5) & (df_temp['date_info'].dt.hour <= 10), 'meal_category'] = '아침'
  df_temp.loc[(df_temp['date_info'].dt.hour > 10) & (df_temp['date_info'].dt.hour <= 16), 'meal_category'] = '점심'
  df_temp.loc[(df_temp['date_info'].dt.hour > 16) & (df_temp['date_info'].dt.hour <= 20), 'meal_category'] = '저녁'
  df_temp.loc[(df_temp['date_info'].dt.hour > 20) | (df_temp['date_info'].dt.hour <= 5), 'meal_category'] = '야식'
  #사용자 전체에 대해 누적 테이블 생성
  df_table = pd.concat([df_table, df_temp],ignore_index=True)

### 타겟 사용자 및 타겟 이미지 지정

In [312]:
#타겟 사용자 지정
user_num_temp = 5 #타겟 사용자의 userID에 따른 순번 입력
#필요시 userID를 input으로 입력받아서도 가능
img_path = path_temp[user_num_temp]
img_path #입력한 userID가 맞는지 확인

'/gdrive/My Drive/Colab Notebooks/사진 자료/1017905'

In [313]:
#평가할 사용자 폴더에 있는 파일 리스트
img_list = np.array(os.listdir(img_path))
img_list = pd.DataFrame(img_list[img_list != '.DS_Store'],columns=['file_name'])
target_image_num = 20 #len(img_list) - 1 # 타겟 이미지 인덱스

#img_list로부터 현재 입력중인 사용자 정보 확인
current_userID = img_path.split('/')
current_userID = current_userID[len(current_userID)-1]
#타겟 사용자의 데이터만 별도로 추출
current_userInfo = df_table.loc[df_table.userID == current_userID]
current_userInfo = current_userInfo.reset_index(drop=True)
current_userInfo['url'] = img_path+'/'+img_list['file_name']
#유사도 분석에 활용할 타겟테이블 생성
target_table = current_userInfo
# 타겟 이미지의 식사분류 지정 및 target과 test(input)테이블 분리
target_mealCat = target_table['meal_category'][target_image_num] 
target_table_input = target_table.drop(index=target_image_num, axis=0)
test_input_df = target_table_input.loc[target_table_input.meal_category == target_mealCat].reset_index(drop=True)
#타겟/비교 이미지 링크 지정
target_image_path = target_table['url'][target_image_num]
input_img_paths = test_input_df['url']
#통합 이미지 링크 지정
integrated_path = np.append(target_image_path,input_img_paths)
#타겟 이미지 링크 확인

In [314]:
target_mealCat

'저녁'

In [206]:
#필요시 타겟 이미지 확인
#PIL.Image.open(str(target_image_url))

### 모델 생성 및 결과 비교

In [297]:
#imageNet을 활용한 inception version 3 사용
module_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

In [275]:
#분류 모델 제작 - 최종적으로 입력벡터와 유사도 반환
def build_graph(module_url, target_image_path):
  # Step 1) Prepare pre-trained model for extracting image features.
  module = hub.KerasLayer(module_url)
  #모델에 적합한 h, w 지정 - 
  height = 229
  width = 229
  CHANNELS = 3
  # Copied a method of https://github.com/GoogleCloudPlatform/cloudml-samples/blob/bf0680726/flowers/trainer/model.py#L181
  # and fixed for all type images (not only jpeg)
  def decode_and_resize(image_str_tensor):
    """Decodes jpeg string, resizes it and returns a uint8 tensor."""
    image = tf.image.decode_image(image_str_tensor, channels=CHANNELS)
    # Note resize expects a batch_size, but tf_map supresses that index,
    # thus we have to expand then squeeze.  Resize returns float32 in the
    # range [0, uint8_max]
    image = tf.expand_dims(image, 0)
    image = tf.compat.v1.image.resize_bilinear(
        image, [height, width], align_corners=False)
    image = tf.squeeze(image)
    image = tf.cast(image, dtype=tf.uint8)
    return image

  def to_img_feature(images):
    """Extract the feature of image vectors"""
    outputs = module(images)
    return outputs

  # Step 2) Extract image features of the target image.
  target_image_bytes = tf.io.gfile.GFile(target_image_path, 'rb').read()
  target_image = tf.constant(target_image_bytes, dtype=tf.string)
  target_image = decode_and_resize(target_image)
  target_image = tf.image.convert_image_dtype(target_image, dtype=tf.float32)
  target_image = tf.expand_dims(target_image, 0)
  target_image = to_img_feature(target_image)

  # Step 3) Extract image features of input images.
  input_byte = tf.compat.v1.placeholder(tf.string, shape=[None])
  input_image = tf.map_fn(decode_and_resize, input_byte, back_prop=False, dtype=tf.uint8)
  input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
  input_image = to_img_feature(input_image)

  # Step 4) Compare cosine_similarities of the target image and the input images.
  dot = tf.tensordot(target_image, tf.transpose(input_image), 1)
  similarity = dot / (tf.norm(target_image, axis=1) * tf.norm(input_image, axis=1))
  similarity = tf.reshape(similarity, [-1])
  
  return input_byte, similarity

In [315]:
#Step5 -- Run
#Compar target and input image

# Load bytes of image files
image_bytes = [open(name, 'rb').read()
               for name in integrated_path]

In [316]:
with tf.Graph().as_default():
  input_byte, similarity_op = build_graph(module_url, target_image_path)

  with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    t0 = time.time() # for time check
    
    # Inference similarities
    similarities = sess.run(similarity_op, feed_dict={input_byte: image_bytes})
    
    print("%d개 이미지 처리시간: %.2f 초" % (len(similarities), time.time() - t0))

27개 이미지 처리시간: 5.98 초


In [317]:
similarities

array([1.0000002 , 0.7167047 , 0.7584221 , 0.7813802 , 0.72129315,
       0.7340472 , 0.69993436, 0.6979532 , 0.7705573 , 0.67411673,
       0.4976635 , 0.7362812 , 0.62610173, 0.68198884, 0.62158376,
       0.5948745 , 0.68202925, 0.70807874, 0.69462067, 0.7176708 ,
       0.74555004, 0.6653539 , 0.69891953, 0.5999698 , 0.5637834 ,
       0.61576045, 0.68794143], dtype=float32)

In [318]:
#실제 출력에 활용한 이미지 리스트 제작
test_input_df['similarity'] = similarities[1:len(similarities)] #타겟사진을 제외한 유사도 추출
test_input_df = test_input_df.sort_values(by=['similarity'],ascending=False).reset_index(drop=True) # 높은 순으로 정렬

sorted_img_path = np.array(test_input_df['url'][0:5]) #상위 n개 추출
sorted_sim = np.array(test_input_df['similarity'][0:5]) #상위 n개 추출

In [ ]:
# 결과 출력
print("### 타겟 이미지 ###")
display(Image(filename = target_image_path, width = 300, height = 300))
print("- 유사도: %.2f" % similarities[0])

print("### 비교 이미지 ###")
for similarity, sorted_img_path in zip(sorted_sim, sorted_img_path):
  display(Image(filename = sorted_img_path, width = 300, height = 300))  
  print("- 유사도: %.2f" % similarity)